In [8]:
import sys

sys.path.append("../..")

In [9]:
import json
import os
from chromadb.config import Settings
from langchain_community.chat_models import ChatOllama
from langchain_chroma import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from constants import MODELS

In [10]:
# Initialize embeddings, vector store, and model
embeddings = HuggingFaceEmbeddings(
    model_name=MODELS.get("embeddings"),
    model_kwargs={"device": "cuda"},
)

setting = Settings(
    anonymized_telemetry=False,
    is_persistent=True,
)

vector_store = Chroma(
    persist_directory="../../.DB_llama3_dulich_dvt",
    embedding_function=embeddings,
    client_settings=setting,
)

retriever = vector_store.as_retriever()

model = ChatOllama(model=MODELS.get("main_llm"))

/home/hungcq/study/AI_Chatbot/venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
docs = retriever.invoke("Đà Nẵng")
docs

[Document(metadata={'Author': '(anonymous)', 'CreationDate': "D:20240909103122-07'00'", 'Creator': '(unspecified)', 'Keywords': '', 'ModDate': "D:20240909103122-07'00'", 'Producer': 'ReportLab PDF Library - www.reportlab.com', 'Subject': '(unspecified)', 'Title': '(anonymous)', 'Trapped': 'False', 'document_name': '/home/hungcq/study/AI_Chatbot/.DOCS/thanhnien_travel_articles.pdf', 'file_path': '/home/hungcq/study/AI_Chatbot/.DOCS/thanhnien_travel_articles.pdf', 'page': 24, 'page_index': 24, 'source': '/home/hungcq/study/AI_Chatbot/.DOCS/thanhnien_travel_articles.pdf', 'topic': 'du lịch', 'total_pages': 64}, page_content='Thời tiết dịp này tại Đà Nẵng trời nắng\nđẹp, rất thuận lợi cho người dân và du khách tham quan, tắm biển, trải nghiệm các dịch\nvụ, tham gia các hoạt động, sự kiện của thành phố. Đặc biệt, lượng khách đến Đà Nẵng\nbằng đường sắt đạt khoảng 10.678 lượt, tăng mạnh 42,43% so với cùng kỳ năm trước.'),
 Document(metadata={'Author': '(anonymous)', 'CreationDate': "D:202409

In [12]:
docs = retriever.invoke("Tại sao không được đi chân trần trên máy bay?")
docs

[Document(metadata={'Author': '(anonymous)', 'CreationDate': "D:20240909103122-07'00'", 'Creator': '(unspecified)', 'Keywords': '', 'ModDate': "D:20240909103122-07'00'", 'Producer': 'ReportLab PDF Library - www.reportlab.com', 'Subject': '(unspecified)', 'Title': '(anonymous)', 'Trapped': 'False', 'document_name': '/home/hungcq/study/AI_Chatbot/.DOCS/thanhnien_travel_articles.pdf', 'file_path': '/home/hungcq/study/AI_Chatbot/.DOCS/thanhnien_travel_articles.pdf', 'page': 0, 'page_index': 0, 'source': '/home/hungcq/study/AI_Chatbot/.DOCS/thanhnien_travel_articles.pdf', 'topic': 'du lịch', 'total_pages': 64}, page_content='Lý do các hãng hàng không Mỹ cấm hành khách đi chân trần trên\nmáy bay\nHành khách đi chân trần trên máy bay ngày càng đông nhưng không phải tất cả các\nhãng hàng không đều chấp nhận, nhiều hãng Mỹ cấm bay nếu khách không đi tất. Để lộ chân trần trên máy bay có thể khiến bạn bị "đá" khỏi một số hãng hàng không Mỹ. Định nghĩa "chân trần" bao gồm cả việc cởi tất khi đã lê

In [13]:
def format_markdown_line(string: str) -> str:
    return f"\n{string}\n".replace("\n", """
""")

def process_questions_from_json(json_path: str, output_md_file: str):
    try:
        # Step 1: Load questions from JSON file
        with open(json_path, "r") as f:
            questions_list = json.load(f)

        # Step 2: Initialize a variable to store previous question's result
        previous_response_content = ""

        # Step 3: Loop through each question in the list
        for index, question_data in enumerate(questions_list):
            question = question_data.get("question")

            # Step 4: Retrieve the documents
            retrieved_docs = retriever.invoke(question)
            context = "\n".join([doc.page_content for doc in retrieved_docs])

            # If there's a previous question, add its result to the context
            if question_data.get("is_next_question") is True:
                context += format_markdown_line(f"Câu trả lời trước đó: {previous_response_content}")

            # Step 5: Generate the final prompt
            prompt_template = PromptTemplate.from_template(
                """
                <s> [INST] Bạn là trợ lý hữu ích, hãy trả lời các câu hỏi về các tài liệu tin tức đã nhập. 
                Chỉ sử dụng ngữ cảnh được cung cấp, không sử dụng bất kỳ thông tin nào ngoài ngữ cảnh này. 
                Nếu bạn không biết hoặc ngữ cảnh không liên quan tới câu hỏi, chỉ cần nói rằng bạn không biết. 
                Bạn chỉ được trả lời bằng tiếng Việt. 
                Các thuật ngữ, địa danh hay cụm từ ngôn ngữ khác có trong tài liệu ngữ cảnh hoặc trong câu hỏi thì không cần dịch sang tiếng Việt.
                [/INST] </s> 
                [INST] Câu hỏi: {question} 
                Ngữ cảnh: {context} 
                Câu trả lời: [/INST]
                """
            )
            prompt = prompt_template.format(question=question, context=context)

            # Step 6: Send the prompt to the model
            response = model.invoke(prompt)

            # Extract model's response content and retrieved document content
            model_response_content = response.content
            retrieved_docs_content = "".join([format_markdown_line(f"* {doc.page_content}") for doc in retrieved_docs])

            # Step 7: Append results to markdown file
            try:
                with open(output_md_file, "a") as md_file:
                    md_file.write(format_markdown_line(f"## Câu hỏi #{index+1}: {question}"))
                    md_file.write(
                        format_markdown_line(f"**Câu trả lời:**\n\n{model_response_content}")
                    )
                    md_file.write(
                        format_markdown_line(
                            f"**Tài liệu được lấy ra:**\n\n{retrieved_docs_content}"
                        )
                    )
            except Exception as e:
                print(f"Error appending to markdown file: {e}")
                continue

            # Step 8: Update previous response content for the next question
            previous_response_content = model_response_content

    except Exception as e:
        print(f"Error processing questions: {e}")

In [14]:
# Example usage
json_path = 'travel_questions.json'
output_md_file = 'travel_results_1.md'
process_questions_from_json(json_path, output_md_file)